<a href="https://colab.research.google.com/github/dsp09/test/blob/master/NER_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import spacy
import random
import spacy.cli

In [ ]:
## Train using Amazon, apple, facebook
df = pd.read_excel('train_3_files.xlsx')
def ner_training(df):
    train_data = [(str(df[col][i]),{"entities":[(0,len(str(df[col][i])),col)]}) for col in df.columns for i in range(len(df))]
    nlp = spacy.blank('en')
    ner = nlp.create_pipe("ner")

    nlp.add_pipe(ner, last=True)

    #ADD THE CUSTOM NAMED ENTITIES HERE
    for i in range(len(df.columns)):
        nlp.entity.add_label(df.columns[i])


    nlp.vocab.vectors.name = 'spacy_pretrained_vectors'
    optimizer = nlp.begin_training()
    for i in range(5):
        random.shuffle(train_data)
        for text, annotations in train_data:
            nlp.update([text], [annotations], sgd=optimizer)
    #SAVE THE CUSTOM NER MODEL TO GOOGLE DRIVE
    nlp.to_disk("custom_ner_model")
    print("Model saved")

In [ ]:
ner_training(df)

Model saved


In [ ]:
## testing the model

In [ ]:
v1 = pd.read_excel('test1.xlsx')

In [ ]:
v1.head()

,Id,first_name,last_name,Status,MedCvg,Admin_month,AnthemAdminMedical
0,SWG222031476,Drace,KALRA,ACTIVE,EMPLOYEE + CHILD(REN),Dec,25.2
1,SWG222028722,Ayvia,RINELLA,ACTIVE,FAMILY,Dec,25.2
2,SWG22205635,Alaynna,NABORS,ACTIVE,FAMILY,Dec,25.2
3,SWG222013575,Roselie,LAGUERRE,ACTIVE,EMPLOYEE ONLY,Dec,25.2
4,SWG22206072,Rana,DRYDEN,ACTIVE,EMPLOYEE + SPOUSE,Dec,25.2


In [ ]:
def ner_standardize(v1):
    nlp2 = spacy.load("custom_ner_model")
    
    def get_col_name(value):
        doc = nlp2(value)
        for ent in doc.ents:
            return ent.label_
    
    def change_col_name(list):
        vals = [get_col_name(x) for x in list]
        maj_val = max(set(vals), key=vals.count)
        return maj_val
    
    def standardize_df(df):
        modified_col_names=[]
        list = [[str(df[col][i]) for i in range(len(df))]for col in df.columns]
        for i in range(len(list)):
            modified_col = change_col_name(list[i])
            modified_col_names.append(modified_col)
        
        df.columns = modified_col_names
        return df
    
    std_v1 = standardize_df(v1)
    
    return (std_v1)

In [ ]:
std_v1 = ner_standardize(v1)

In [ ]:
std_v1.head()

,EmployeeID,First_name,Last_name,Status,Medical_coverge,Admin_month,AnthemAdminMedical
0,SWG222031476,Drace,KALRA,ACTIVE,EMPLOYEE + CHILD(REN),Dec,25.2
1,SWG222028722,Ayvia,RINELLA,ACTIVE,FAMILY,Dec,25.2
2,SWG22205635,Alaynna,NABORS,ACTIVE,FAMILY,Dec,25.2
3,SWG222013575,Roselie,LAGUERRE,ACTIVE,EMPLOYEE ONLY,Dec,25.2
4,SWG22206072,Rana,DRYDEN,ACTIVE,EMPLOYEE + SPOUSE,Dec,25.2


In [ ]:
## Google
v2= pd.read_excel('test1.xlsx')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
v2.head()

,SSN_ID,FIRST_NAME_,LAST_NAME_,Cost_MEDICAL,Med_Cov,Month,Status
0,SWG222068304,Francoise,Rautenstrauch,28.95,EMPLOYEE ONLY,dec,ACTIVE
1,SWG222032304,Kendra,Loud,28.95,EMPLOYEE + SPOUSE,dec,ACTIVE
2,SWG222077108,Lourdes,Bauswell,28.95,EMPLOYEE + SPOUSE,dec,ACTIVE
3,SWG222011607,Hannah,Edmison,28.95,FAMILY,dec,ACTIVE
4,SWG222072443,Tom,Loeza,28.95,EMPLOYEE ONLY,dec,ACTIVE


In [ ]:
std_v2 = ner_standardize(v2)

In [ ]:
std_v2.head()

,EmployeeID,First_name,Last_name,AnthemAdminMedical,Medical_coverge,First_name,Status
0,SWG222068304,Francoise,Rautenstrauch,28.95,EMPLOYEE ONLY,dec,ACTIVE
1,SWG222032304,Kendra,Loud,28.95,EMPLOYEE + SPOUSE,dec,ACTIVE
2,SWG222077108,Lourdes,Bauswell,28.95,EMPLOYEE + SPOUSE,dec,ACTIVE
3,SWG222011607,Hannah,Edmison,28.95,FAMILY,dec,ACTIVE
4,SWG222072443,Tom,Loeza,28.95,EMPLOYEE ONLY,dec,ACTIVE


In [ ]:
## Tesla
tesla = pd.read_csv('tesla_vendor.csv')
tesla.head()

,MemberID,First Name,Last Name,Status,MedCvg,AdminMonth,AnthemAdminMedical
0,SWG222017831,Julionna,SHEATS,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222028601,Araminta,MILLA,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG222024656,Otavio,POPPER,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG222027740,Korede,SURIANO,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG22203443,Olin,MCMURRAY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
std_tesla = ner_standardize(tesla)
std_tesla.head()

,EmployeeID,First_name,Last_name,Status,Medical_coverge,Admin_month,AnthemAdminMedical
0,SWG222017831,Julionna,SHEATS,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222028601,Araminta,MILLA,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG222024656,Otavio,POPPER,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG222027740,Korede,SURIANO,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG22203443,Olin,MCMURRAY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
## Legato
legato = pd.read_excel('legato_vendor.xlsx')
legato.head()

,FN,LN,EMPID,Coverage,Month,status,Amount_anthem
0,EMMA,SMITH,SWG22207081,FAMILY,DEC,active,25.66
1,OLIVIA,JOHNSON,SWG22201389,FAMILY,DEC,active,25.66
2,NOAH,WILLIAMS,SWG22204421,EMPLOYEE + SPOUSE,DEC,active,25.66
3,LIAM,BROWN,SWG222017801,EMPLOYEE + CHILD(REN),DEC,active,25.66
4,SOPHIA,JONES,SWG22207727,EMPLOYEE + CHILD(REN),DEC,active,25.66


In [ ]:
std_legato = ner_standardize(legato)
std_legato.head()

,Last_name,Last_name,EmployeeID,Medical_coverge,Admin_month,First_name,First_name
0,EMMA,SMITH,SWG22207081,FAMILY,DEC,active,25.66
1,OLIVIA,JOHNSON,SWG22201389,FAMILY,DEC,active,25.66
2,NOAH,WILLIAMS,SWG22204421,EMPLOYEE + SPOUSE,DEC,active,25.66
3,LIAM,BROWN,SWG222017801,EMPLOYEE + CHILD(REN),DEC,active,25.66
4,SOPHIA,JONES,SWG22207727,EMPLOYEE + CHILD(REN),DEC,active,25.66


In [ ]:
## Norton
norton = pd.read_excel('norton_vendor.xlsx')
norton.head()

,ssn,name,surname,admin_medical,med_coverage,month,status
0,SWG222016090,Jaime,LANG,25.66,FAMILY,dec,ACTIVE
1,SWG222028054,Aileen,PRATT,25.66,EMPLOYEE + SPOUSE,dec,ACTIVE
2,SWG22203963,Lennon,LARA,25.66,EMPLOYEE + SPOUSE,dec,ACTIVE
3,SWG222032199,Charley,BROCK,25.66,EMPLOYEE + CHILD(REN),dec,ACTIVE
4,SWG222015176,Rosemary,BALLARD,25.66,FAMILY,dec,ACTIVE


In [ ]:
std_norton = ner_standardize(norton)
std_norton.head()

,EmployeeID,First_name,Last_name,First_name,Medical_coverge,Admin_month,Status
0,SWG222016090,Jaime,LANG,25.66,FAMILY,dec,ACTIVE
1,SWG222028054,Aileen,PRATT,25.66,EMPLOYEE + SPOUSE,dec,ACTIVE
2,SWG22203963,Lennon,LARA,25.66,EMPLOYEE + SPOUSE,dec,ACTIVE
3,SWG222032199,Charley,BROCK,25.66,EMPLOYEE + CHILD(REN),dec,ACTIVE
4,SWG222015176,Rosemary,BALLARD,25.66,FAMILY,dec,ACTIVE


In [ ]:
## MG
mg = pd.read_csv('morgan_stanly_vendor.csv')
mg.head()

,MemberID,First Name,Last Name,Status,MedCvg,AdminMonth,AnthemAdminMedical
0,SWG222023700,Shalayah,FORSHEY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222022026,Kaise,MCGARRITY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG222010850,Vivan,HOPWOOD,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG22204551,Marquez,BRILEY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG222031729,Isse,SANSOUCIE,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
std_mg = ner_standardize(mg)
std_mg.head()

,EmployeeID,First_name,Last_name,Status,Medical_coverge,Admin_month,AnthemAdminMedical
0,SWG222023700,Shalayah,FORSHEY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222022026,Kaise,MCGARRITY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG222010850,Vivan,HOPWOOD,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG22204551,Marquez,BRILEY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG222031729,Isse,SANSOUCIE,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
## Microsoft
microsoft = pd.read_csv('microsoft_vendor.csv')
microsoft.head()

,MemberID,First Name,Last Name,Status,MedCvg,AdminMonth,AnthemAdminMedical
0,SWG222019511,Nuala,NETHERY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222030921,Zadey,STUMPH,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG222028335,Aalisa,KRASKA,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG22203594,Joan,RAFFERTY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG22209886,Aiko,SHUEY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
std_microsoft = ner_standardize(microsoft)
std_microsoft.head()

,EmployeeID,First_name,Last_name,Status,Medical_coverge,Admin_month,AnthemAdminMedical
0,SWG222019511,Nuala,NETHERY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222030921,Zadey,STUMPH,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG222028335,Aalisa,KRASKA,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG22203594,Joan,RAFFERTY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG22209886,Aiko,SHUEY,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
## Kpmg
kpmg = pd.read_csv('kpmg_vendor.csv')
kpmg.head()

,MemberID,First Name,Last Name,Status,MedCvg,AdminMonth,AnthemAdminMedical
0,SWG222021921,Heyden,HAZELL,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222024568,Maxin,MANVILLE,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG22201024,Cameron,COSTELLO,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG2220646,Ronan,BAXTER,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG222019785,Arshaan,BELUE,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2


In [ ]:
std_kpmg = ner_standardize(kpmg)
std_kpmg.head()

,EmployeeID,First_name,Last_name,Status,Medical_coverge,Admin_month,AnthemAdminMedical
0,SWG222021921,Heyden,HAZELL,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
1,SWG222024568,Maxin,MANVILLE,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
2,SWG22201024,Cameron,COSTELLO,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
3,SWG2220646,Ronan,BAXTER,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
4,SWG222019785,Arshaan,BELUE,ACTIVE,EMPLOYEE+SPOUSE,Dec,25.2
